In [49]:
import os
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.cloud import bigquery
from google.cloud import dataplex
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.cloud import storage


def getInfoFromPage(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    paraminfo = soup.find_all("b", "parameters__value")

    title = soup.find("h1", "sticker__title")
    if (title != None): title = title.text.strip()
    else: title = "NULL"

    if (len(paraminfo) >= 1):
        city = re.sub(" +", " ", paraminfo[0].text.strip())
    else: city = "NULL"

    if (len(paraminfo) >= 2):
        square = paraminfo[1].text.strip()
    else: square = "NULL"

    if (len(paraminfo) >= 3):
        rooms = paraminfo[2].text.strip()
    else: rooms = "NULL"

    if (len(paraminfo) >= 4):
        floor = paraminfo[3].text.strip()
    else: floor = "NULL"

    if (len(paraminfo) >= 5):
        for text in paraminfo:
            builtIn = re.findall("(?:[1][9][0-9][0-9]|[2][0][0-2][0-9])", text.text)
            if (builtIn == []): builtIn = "NULL"
            else:
                builtIn = builtIn[0]
                break
    else: builtIn = "NULL"

    paraminfo = soup.find_all("div", "parameters__value")

    if (len(paraminfo) >= 1):
        updateTime = paraminfo[0].text.strip()
    else: updateTime = "NULL"

    if (len(paraminfo) >= 1):
        uploadTime = paraminfo[1].text.strip()
    else: uploadTime = "NULL"

    price = soup.find("span", "priceInfo__value")
    if (price != None): price = re.sub(" +", " ", soup.find("span", "priceInfo__value").text.replace("\n", '').strip())
    else: price = "NULL"

    pricePerMeter = soup.find("span", "priceInfo__additional")
    if (pricePerMeter != None): pricePerMeter = pricePerMeter.text.strip()
    else: pricePerMeter = "NULL"

    description = soup.find("div", "description__container")
    if (description != None): description = description.text.replace("\n", '').replace("\r", '').strip()
    else: description = "NULL"

    return (title, city, square, rooms, floor, builtIn, updateTime, uploadTime, price, pricePerMeter, description, url)

def getLinksFromPage(url):
    links = []
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    divs = soup.find_all("div", "listing__teaserWrapper")
    for div in divs:
        links.append(div.find('a', "teaserLink")['href'])
    return links

def getMaxPage(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup.find('a', "")

df = pd.DataFrame(columns=['Title', 'City', 'Square', 'Rooms', 'Floor', 'Built In', 'Update time', 'Upload time', 'Price',
                           'Price per meter', 'Description', 'Link'])

links = []
URL = "https://gratka.pl/nieruchomosci/mieszkania?page="
MaxPage = 2

for i in range(1, MaxPage):
    links = links + getLinksFromPage(URL + str(i))
    print("Ładowanie strony " + str(i) + "/" + str(MaxPage - 1))

for n, link in enumerate(links):
    df.loc[n] = getInfoFromPage(link)
    print("Pobieranie informacji ze strony " + str(n + 1) + "/" + str(len(links)))

# Сохранение в CSV
df.to_csv(r"output1.csv", index=True, header=True)
print("Zapisane do output1.plik csv")

# Преобразование в JSON
df_json = df.to_json(orient='records')
with open('output1.json', 'w') as json_file:
    json_file.write(df_json)
print("Zapisane do output1.plik json")


def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f'File {source_file_name} uploaded to {destination_blob_name} in {bucket_name}.')

source_file_name = 'output1.csv'

#############

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key.json'

storage_client = storage.Client()
bucket_name = 'gratka_bucket'

# Проверяем, существует ли ведро
bucket = storage_client.bucket(bucket_name)
if not bucket.exists():
    bucket = storage_client.create_bucket(bucket, location='EU')

def upload_to_bucket(blob_name, file_path, bucket_name):
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(file_path)
    return True


file_path = r'C:\Users\lukum\PycharmProjects\pythonProject\output1.csv'
upload_to_bucket('output1.csv', file_path, 'gratka_bucket')

def clear_table(client, dataset_name, table_name):
    """Удаляет все строки из таблицы BigQuery."""
    dataset_ref = client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)

    # Получаем объект таблицы
    table = client.get_table(table_ref)

    # Формируем SQL-запрос для очистки таблицы (используя TRUNCATE TABLE)
    sql = f"TRUNCATE TABLE `{table.project}.{table.dataset_id}.{table.table_id}`"

    # Выполняем запрос
    query_job = client.query(sql)
    query_job.result()  # Ждем завершения запроса

    print(f"aktualizacja tabeli... {table.table_id}.")

def upload_to_bigquery(bucket_name, source_file_name, dataset_name, table_name):
    """Загружает CSV-файл из Cloud Storage в таблицу BigQuery."""
    # Создаем экземпляр клиента BigQuery
    client = bigquery.Client()

    # Очищаем таблицу
    clear_table(client, dataset_name, table_name)

    # Определяем URI источника и таблицы назначения
    source_uri = f"gs://{bucket_name}/{source_file_name}"
    destination_table = f"{client.project}.{dataset_name}.{table_name}"

    # Загружаем файл в таблицу BigQuery
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,  # Пропускаем заголовок
        field_delimiter=',',  # Можно изменить разделитель полей, если он отличается
    )
    load_job = client.load_table_from_uri(
        source_uri,
        destination_table,
        job_config=job_config
    )

    # Ожидаем завершения загрузки
    load_job.result()

    print(f"dane {source_file_name} przesłane do tablicy BigQuery {destination_table}.")

# Пример использования:
upload_to_bigquery(bucket_name, "output1.csv", "gratka", "gratka")

Ładowanie strony 1/1
Pobieranie informacji ze strony 1/32
Pobieranie informacji ze strony 2/32
Pobieranie informacji ze strony 3/32
Pobieranie informacji ze strony 4/32
Pobieranie informacji ze strony 5/32
Pobieranie informacji ze strony 6/32
Pobieranie informacji ze strony 7/32
Pobieranie informacji ze strony 8/32
Pobieranie informacji ze strony 9/32
Pobieranie informacji ze strony 10/32
Pobieranie informacji ze strony 11/32
Pobieranie informacji ze strony 12/32
Pobieranie informacji ze strony 13/32
Pobieranie informacji ze strony 14/32
Pobieranie informacji ze strony 15/32
Pobieranie informacji ze strony 16/32
Pobieranie informacji ze strony 17/32
Pobieranie informacji ze strony 18/32
Pobieranie informacji ze strony 19/32
Pobieranie informacji ze strony 20/32
Pobieranie informacji ze strony 21/32
Pobieranie informacji ze strony 22/32
Pobieranie informacji ze strony 23/32
Pobieranie informacji ze strony 24/32
Pobieranie informacji ze strony 25/32
Pobieranie informacji ze strony 26/32
